In [1]:
import numpy as np 
import pandas as pd
import string
import ast
import nltk

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credit = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
data= pd.merge(movies,credit,on="title")

In [4]:
movies_data = data[["movie_id","title","overview","genres","keywords","cast","crew"]]

In [5]:
movies_data = movies_data.dropna()


In [6]:
def convert(obj):
    genres = []
    for item in ast.literal_eval(obj):
        genres.append(item["name"])
    return genres

In [7]:
def convert_cast(obj):
    cast_total = ast.literal_eval(obj)
    cast = []
    for i in range(0,3):
        try:
            cast.append(cast_total[i]["name"])
        except:
            continue
    return cast

In [8]:
def convert_crew(obj):
    crew = []
    for item in ast.literal_eval(obj):
        if item["job"]== 'Director':
            crew.append(item["name"])
            break
    return crew

In [9]:
movies_data["genres"] = movies_data["genres"].apply(convert)

In [10]:
movies_data["keywords"] = movies_data["keywords"].apply(convert)

In [11]:
movies_data["cast"] = movies_data["cast"].apply(convert_cast)

In [12]:
movies_data["crew"] = movies_data["crew"].apply(convert_crew)       

In [13]:
movies_data["overview"] = movies_data["overview"].apply(lambda x:x.split())

In [14]:
movies_data["genres"] = movies_data["genres"].apply(lambda x:[ i.replace(" ","") for i in x])
movies_data["keywords"] = movies_data["keywords"].apply(lambda x:[ i.replace(" ","") for i in x])
movies_data["cast"] = movies_data["cast"].apply(lambda x:[ i.replace(" ","") for i in x])
movies_data["crew"] = movies_data["crew"].apply(lambda x:[ i.replace(" ","") for i in x])

In [15]:
movies_data["tag"] =movies_data["genres"]+ movies_data["keywords"]+ movies_data["cast"]+ movies_data["crew"] +movies_data["overview"]

In [16]:
new_movies_data = movies_data[["movie_id","title","tag"]]

In [17]:
new_movies_data["tag"] = new_movies_data["tag"].apply(lambda x: " ".join(x))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14884\1215490741.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies_data["tag"] = new_movies_data["tag"].apply(lambda x: " ".join(x))


In [18]:
 new_movies_data["tag"]= new_movies_data["tag"].apply(lambda x: x.lower())

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14884\1527787609.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies_data["tag"]= new_movies_data["tag"].apply(lambda x: x.lower())


In [19]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [20]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
    

In [21]:
new_movies_data["tag"] = new_movies_data["tag"].apply(stem)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14884\2097464694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies_data["tag"] = new_movies_data["tag"].apply(stem)


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [23]:
vector  =  cv.fit_transform(new_movies_data["tag"]).toarray()

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
similarity  = cosine_similarity(vector)

In [36]:
def recommend(movie):
    index = new_movies_data[new_movies_data['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_movies_data.iloc[i[0]].title)
        

In [37]:
recommend('Batman')

Batman
Batman & Robin
Batman Begins
Batman Returns
The R.M.


In [27]:
import pickle

In [28]:
pickle.dump(new_movies_data,open("movie_data.pkl","wb"))
pickle.dump(similarity,open("similarity.pkl","wb"))

In [42]:
new_movies_data.iloc[0].movie_id

19995